In [337]:
import pandas as pd

def translate_team(team_name):
    # Check if translation exists, otherwise return the original name
    return translation_dict.get(team_name, f"Unknown ({team_name})")

Head_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\prediction_data/'
# Step 1: Load the prediction CSV
# prediction_df = pd.read_csv(Head_path+'prediction.csv')
prediction_df = pd.read_excel(Head_path+'prediction.xlsx')

# Step 2: Load the translation CSV (assuming it has 'Chinese' and 'English' columns)
translation_df = pd.read_csv(Head_path+'translation.csv', encoding='utf-8')

# Step 3: Create a dictionary for mapping Chinese team names to English
translation_dict = pd.Series(translation_df['English'].values, index=translation_df['Chinese'].values).to_dict()

# Step 3: Create a dictionary for mapping Chinese aHh to Math
translation_dict_math = pd.Series(translation_df['Math_Hh'].values, index=translation_df['Chinese_Hh'].values).to_dict()

# Step 4: Replace Chinese team names with English in HomeTeam and AwayTeam columns
prediction_df['HomeTeam'] = prediction_df['HomeTeam'].map(translation_dict).fillna(prediction_df['HomeTeam'])
prediction_df['AwayTeam'] = prediction_df['AwayTeam'].map(translation_dict).fillna(prediction_df['AwayTeam'])
prediction_df['AHh'] = prediction_df['AHh'].map(translation_dict_math).fillna(prediction_df['AHh'])
# prediction_df['HomeTeam'] = prediction_df['HomeTeam'].apply(translate_team)
# prediction_df['AwayTeam'] = prediction_df['AwayTeam'].apply(translate_team)

# Step 5: Save the updated dataframe (optional)
prediction_df.to_csv(Head_path+'updated_prediction.csv', index=False, encoding='utf-8-sig')
print(prediction_df)

   Unnamed: 0  Div       Date    HomeTeam    AwayTeam  B365H  B365D  B365A  \
0           3   D1 2025-05-09   Wolfsburg  Hoffenheim   2.30   3.60    2.8   
1           4   I1 2025-05-09       Milan     Bologna   2.30   3.40    3.0   
2           6  SP1 2025-05-09  Las Palmas   Vallecano   2.35   3.25    3.1   

    AHh  B365AHH  ...  balance_val_model  model_v4  info  FTR_real  \
0 -0.25     2.08  ...                NaN       NaN   NaN       NaN   
1 -0.25     2.02  ...                NaN       NaN   NaN       NaN   
2 -0.25     2.03  ...                NaN       NaN   NaN       NaN   

   predict1.1  predict2.1  predict3.1  model_FTR  info_FTR  match_div_info  
0         NaN         NaN         NaN        NaN       NaN             NaN  
1         NaN         NaN         NaN        NaN       NaN             NaN  
2         NaN         NaN         NaN        NaN       NaN             NaN  

[3 rows x 33 columns]


In [338]:
# import os
# 
# # 文件夹路径
# folder_path = 'D:\Projects\Other_exp\Jupyter\orie4741-fp\elo/'
# 
# # 获取文件夹下所有文件的名称
# file_names = []
# for root, dirs, files in os.walk(folder_path):
#     for file in files:
#         # 分离文件名和扩展名，获取文件名部分
#         file_name_without_extension = os.path.splitext(file)[0]
#         file_names.append(file_name_without_extension)
# 
# # 将文件名称写入到 txt 文件，按行存储
# with open(folder_path + 'file_names.txt', 'w', encoding='utf-8') as f:
#     for file_name in file_names:
#         f.write(file_name + '\n')
# 
# print("文件名称已成功保存到 file_names.txt")

In [339]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import joblib
from joblib import load, dump
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [340]:
df = pd.read_csv('prediction_data/updated_prediction.csv', low_memory = False)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,balance_val_model,model_v4,info,FTR_real,predict1.1,predict2.1,predict3.1,model_FTR,info_FTR,match_div_info
0,3,D1,2025-05-09,Wolfsburg,Hoffenheim,2.30,3.60,2.8,-0.25,2.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,I1,2025-05-09,Milan,Bologna,2.30,3.40,3.0,-0.25,2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,SP1,2025-05-09,Las Palmas,Vallecano,2.35,3.25,3.1,-0.25,2.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [341]:
conditions = [
    df['AHh'] >= 2.5,         # AHh >= 2.75
    (df['AHh'] >= 1.5) & (df['AHh'] <= 2.25),  # 1.75 < AHh <= 2.75
    (df['AHh'] >= 0.25) & (df['AHh'] <= 1.25),  # 0.25 < AHh <= 1.75
    df['AHh'] == 0,            # AHh == 0
    (df['AHh'] >= -1.25) & (df['AHh'] <= -0.25),  # -1.75 < AHh <= -0.25
    (df['AHh'] >= -2.25) & (df['AHh'] <= -1.5),  # -2.75 < AHh <= -1.75
    df['AHh'] <= -2.5
]
labels = [3, 2, 1, 0, -1, -2, -3]

df['balance_val'] = np.select(conditions, labels)
df

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,...,balance_val_model,model_v4,info,FTR_real,predict1.1,predict2.1,predict3.1,model_FTR,info_FTR,match_div_info
0,3,D1,2025-05-09,Wolfsburg,Hoffenheim,2.30,3.60,2.8,-0.25,2.08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,I1,2025-05-09,Milan,Bologna,2.30,3.40,3.0,-0.25,2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,SP1,2025-05-09,Las Palmas,Vallecano,2.35,3.25,3.1,-0.25,2.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "B365H", "B365D", "B365A","AHh", "B365AHH", "B365AHA", "balance_val"
                    ]
df = df[features_to_keep]

In [343]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    # if date > '2000-05-01':
    #     aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("prediction_data/updated_prediction_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,D1,2025-05-09,Wolfsburg,Hoffenheim,2.30,3.60,2.8,-0.25,2.08,1.82,-1,1638.784424,1608.671387
1,I1,2025-05-09,Milan,Bologna,2.30,3.40,3.0,-0.25,2.02,1.88,-1,1789.167236,1791.398926
2,SP1,2025-05-09,Las Palmas,Vallecano,2.35,3.25,3.1,-0.25,2.03,1.83,-1,1600.078125,1645.527100


In [344]:
# 提前9个月，用来划分赛季
# 将字符串转换为日期类型，指定日期格式
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Date'] = df['Date'] - pd.DateOffset(months=6)
df

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,D1,2024-11-09,Wolfsburg,Hoffenheim,2.30,3.60,2.8,-0.25,2.08,1.82,-1,1638.784424,1608.671387
1,I1,2024-11-09,Milan,Bologna,2.30,3.40,3.0,-0.25,2.02,1.88,-1,1789.167236,1791.398926
2,SP1,2024-11-09,Las Palmas,Vallecano,2.35,3.25,3.1,-0.25,2.03,1.83,-1,1600.078125,1645.527100


In [345]:
df_noNa = df.dropna()
df_noNa.reset_index(inplace=True, drop=True)
df_noNa

,Div,Date,HomeTeam,AwayTeam,B365H,B365D,B365A,AHh,B365AHH,B365AHA,balance_val,HomeTeamELO,AwayTeamELO
0,D1,2024-11-09,Wolfsburg,Hoffenheim,2.30,3.60,2.8,-0.25,2.08,1.82,-1,1638.784424,1608.671387
1,I1,2024-11-09,Milan,Bologna,2.30,3.40,3.0,-0.25,2.02,1.88,-1,1789.167236,1791.398926
2,SP1,2024-11-09,Las Palmas,Vallecano,2.35,3.25,3.1,-0.25,2.03,1.83,-1,1600.078125,1645.527100


In [346]:
# df_noNa_new = df_noNa.iloc[:3]
df_noNa_new = df_noNa

In [347]:
# 与历史数据合并，计算后续特征
history_df = pd.read_csv(Head_path+'history_data_balance.csv')
history_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003-02-02,Auxerre,Nice,1,2,A,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,0,-1
1,F1,2003-02-02,Guingamp,Marseille,0,1,A,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0
2,D1,2003-02-02,Hamburg,Hannover,0,3,A,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,0,-1
3,D1,2003-02-02,Hertha,Werder Bremen,0,3,A,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,0,-1
4,F1,2003-02-02,Lens,Le Mans,0,0,D,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38507,SP1,2024-11-04,Sociedad,Ath Bilbao,0,0,D,2.550,3.000,3.000,-0.25,2.100,1.700,1683.467285,1797.882568,0,-1
38508,E0,2024-11-04,West Ham,Tottenham,1,1,D,2.100,3.700,3.250,-0.25,1.750,2.050,1729.567261,1790.107788,0,-1
38509,E0,2024-11-05,Crystal Palace,Nott'm Forest,1,1,D,2.250,3.400,3.200,-0.25,1.930,1.850,1807.064453,1802.088501,0,-1
38510,I1,2024-11-05,Genoa,Milan,1,2,A,4.750,3.700,1.800,0.75,1.800,1.980,1656.094849,1783.127075,1,1


In [348]:
# 合并两个 DataFrame，重置索引
df_noNa_comb = pd.concat([history_df, df_noNa_new], ignore_index=True)
df_noNa_comb

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,easy_label,balance_val
0,F1,2003-02-02,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,0.0,-1
1,F1,2003-02-02,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0.0,0
2,D1,2003-02-02,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,0.0,-1
3,D1,2003-02-02,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,0.0,-1
4,F1,2003-02-02,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,0.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,I1,2024-11-05,Genoa,Milan,1.0,2.0,A,4.750,3.700,1.800,0.75,1.800,1.980,1656.094849,1783.127075,1.0,1
38511,SP1,2024-11-05,Girona,Mallorca,1.0,0.0,H,2.000,3.100,4.200,-0.50,2.050,1.850,1652.915283,1660.488281,1.0,-1
38512,D1,2024-11-09 00:00:00,Wolfsburg,Hoffenheim,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,2.080,1.820,1638.784424,1608.671387,NaN,-1
38513,I1,2024-11-09 00:00:00,Milan,Bologna,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,2.020,1.880,1789.167236,1791.398926,NaN,-1


In [349]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [350]:
# # 加载编码器
div_encoder = joblib.load('div_encoder.pkl')
home_encoder = joblib.load('home_encoder.pkl')
away_encoder = joblib.load('away_encoder.pkl')
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder
# 
# div_encoder = OneHotEncoder()
# home_encoder = OneHotEncoder()
# away_encoder = OneHotEncoder()
# 你可以使用加载的编码器进行转换
onehot_div = div_encoder.transform(df_noNa_comb.Div.values.reshape(-1,1)).toarray().astype(int)
onehot_div_df = pd.DataFrame(onehot_div, columns = ["Div "+str(int(i)) for i in range(onehot_div.shape[1])])

onehot_home = home_encoder.transform(df_noNa_comb.HomeTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_home_df = pd.DataFrame(onehot_home, columns = ['HomeTeam ' + str(int(i)) for i in np.arange(onehot_home.shape[1])])

onehot_away = away_encoder.transform(df_noNa_comb.AwayTeam.values.reshape(-1,1)).toarray().astype(int)
onehot_away_df = pd.DataFrame(onehot_away, columns = ['AwayTeam ' + str(int(i)) for i in np.arange(onehot_away.shape[1])])

df_noNa_div = pd.concat([df_noNa_comb, onehot_div_df, onehot_home_df, onehot_away_df], axis = 1)
df_noNa_div.drop(columns = ['Div'], inplace = True)
df_noNa_div

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,...,AwayTeam 197,AwayTeam 198,AwayTeam 199,AwayTeam 200,AwayTeam 201,AwayTeam 202,AwayTeam 203,AwayTeam 204,AwayTeam 205,AwayTeam 206
0,2003-02-02,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,-0.75,...,0,0,0,0,0,0,0,0,0,0
1,2003-02-02,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,0.00,...,0,0,0,0,0,0,0,0,0,0
2,2003-02-02,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,-0.75,...,0,0,0,0,0,0,0,0,0,0
3,2003-02-02,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,-0.75,...,0,0,1,0,0,0,0,0,0,0
4,2003-02-02,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,-0.75,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,2024-11-05,Genoa,Milan,1.0,2.0,A,4.750,3.700,1.800,0.75,...,0,0,0,0,0,0,0,0,0,0
38511,2024-11-05,Girona,Mallorca,1.0,0.0,H,2.000,3.100,4.200,-0.50,...,0,0,0,0,0,0,0,0,0,0
38512,2024-11-09 00:00:00,Wolfsburg,Hoffenheim,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,...,0,0,0,0,0,0,0,0,0,0
38513,2024-11-09 00:00:00,Milan,Bologna,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,...,0,0,0,0,0,0,0,0,0,0


In [351]:
# 处理未进行比赛的easy_label
df_noNa_div['easy_label'] = df_noNa_div['easy_label'].fillna(1)  # 或其他有意义的值

In [352]:
target_encoder = joblib.load('target_encoder.pkl')
df_noNa_div['Result'] = target_encoder.transform(df_noNa_div.easy_label) 

In [353]:
df_noNa_div['Year'] = pd.DatetimeIndex(df_noNa_div.Date).year

df_noNa_div['Month'] = pd.DatetimeIndex(df_noNa_div.Date).month
df_noNa_div['Sin_Month'] = np.sin(2*np.pi*df_noNa_div.Month/12)
df_noNa_div['Cos_Month'] = np.cos(2*np.pi*df_noNa_div.Month/12)

df_noNa_div['DayofYear'] = pd.DatetimeIndex(df_noNa_div.Date).dayofyear
df_noNa_div['Sin_Day'] = np.sin(2*np.pi*df_noNa_div.DayofYear/365)
df_noNa_div['Cos_Day'] = np.cos(2*np.pi*df_noNa_div.DayofYear/365)

# 注意 inplace是在原始frame修改，返回值是Nonetype
# learning_df = learning_df_div.drop(columns = ['Date','Month'], inplace = True)
# df_noNa_div_date = df_noNa_div.drop(columns = ['Date','Month'])
df_noNa_div_date = df_noNa_div.drop(columns = ['Date'])
df_noNa_div_date

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,...,AwayTeam 205,AwayTeam 206,Result,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,-0.75,2.050,...,0,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,0.00,1.925,...,0,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,-0.75,1.800,...,0,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,-0.75,2.025,...,0,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,-0.75,1.900,...,0,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,Genoa,Milan,1.0,2.0,A,4.750,3.700,1.800,0.75,1.800,...,0,0,1,2024,11,-0.500000,0.866025,310,-0.811539,0.584298
38511,Girona,Mallorca,1.0,0.0,H,2.000,3.100,4.200,-0.50,2.050,...,0,0,1,2024,11,-0.500000,0.866025,310,-0.811539,0.584298
38512,Wolfsburg,Hoffenheim,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,2.080,...,0,0,1,2024,11,-0.500000,0.866025,314,-0.769415,0.638749
38513,Milan,Bologna,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,2.020,...,0,0,1,2024,11,-0.500000,0.866025,314,-0.769415,0.638749


In [354]:
# 定义一个函数来计算两队之间上一场比赛的结果
def compute_last_matches(df):
    
    unique_matchups = list(set((list(zip(df.HomeTeam, df.AwayTeam)))))
    df['Last Match Result'] = np.nan
    for home, away in unique_matchups:
        matchup_df = df[(df.HomeTeam == home) & (df.AwayTeam == away)]
        # 使用 shift(1) 方法将 FTR（全场比赛结果）列中的数据向下移动一行，这样每行的 last_match_result 将对应于这两队之前的一场比赛的结果。fill_value='Na' 确保了数据移动后空出的位置填充为 'Na'。
        # last_match_result = matchup_df.FTR.shift(1, fill_value='Na')
        last_match_result = matchup_df.Result.shift(1, fill_value=3)
        df.loc[matchup_df.index, 'Last Match Result'] = last_match_result
        
    lmr_encoder = LabelEncoder()
    df['Last Match Result'] = lmr_encoder.fit_transform(df['Last Match Result'])
    df.drop(columns = ['easy_label'], inplace = True)
    return df
df_noNa_div_date_lastMatch = compute_last_matches(df_noNa_div_date)
df_noNa_div_date_lastMatch

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,...,AwayTeam 206,Result,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,-0.75,2.050,...,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942,3
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,0.00,1.925,...,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942,3
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,-0.75,1.800,...,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942,3
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,-0.75,2.025,...,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942,3
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,-0.75,1.900,...,0,0,2003,2,0.866025,0.500000,33,0.538005,0.842942,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,Genoa,Milan,1.0,2.0,A,4.750,3.700,1.800,0.75,1.800,...,0,1,2024,11,-0.500000,0.866025,310,-0.811539,0.584298,1
38511,Girona,Mallorca,1.0,0.0,H,2.000,3.100,4.200,-0.50,2.050,...,0,1,2024,11,-0.500000,0.866025,310,-0.811539,0.584298,2
38512,Wolfsburg,Hoffenheim,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,2.080,...,0,1,2024,11,-0.500000,0.866025,314,-0.769415,0.638749,0
38513,Milan,Bologna,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,2.020,...,0,1,2024,11,-0.500000,0.866025,314,-0.769415,0.638749,0


In [355]:
def compute_winstreak(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinStreak'] = None
        year_df['AwayWinStreak'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        # if year > 2024:
        #     # 将 AwayWin = 3 当作 0 来处理，保持计算连胜记录
        #     year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
        #     year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            home_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(home_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['HomeWin'] == 1:  # 如果上一场比赛主队赢
                    home_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    home_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['HomeWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
            # team_grouper = (team_df.HomeWin != team_df.HomeWin.shift()).cumsum()
            # team_df['HomeWinStreak'] = team_df[['HomeWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.HomeWinStreak >0, 'HomeWinStreak'] -= 1
            # year_df.loc[team_df.index, 'HomeWinStreak'] = team_df.HomeWinStreak
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

            away_win_streak = 0  # 初始化连胜场数
            streaks = []  # 用来存储每场比赛的连续胜利次数
            for idx, row in team_df.iterrows():
                streaks.append(away_win_streak)  # 当前场次视为未进行，记录上一场的连胜次数
                # 计算当前场次的连胜，忽略当前比赛的胜负
                if row['AwayWin'] == 1:  # 如果上一场比赛主队赢
                    away_win_streak += 1  # 连胜场数递增
                else:  # 如果上一场比赛主队输了
                    away_win_streak = 0  # 连胜场数重置为 0
            # 将计算出的连胜场数赋值到 DataFrame 中
            team_df['AwayWinStreak'] = streaks
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
            # team_grouper = (team_df.AwayWin != team_df.AwayWin.shift()).cumsum()
            # team_df['AwayWinStreak'] = team_df[['AwayWin']].groupby(team_grouper).cumsum()
            # team_df.loc[team_df.AwayWinStreak >0, 'AwayWinStreak'] -= 1
            # year_df.loc[team_df.index, 'AwayWinStreak'] = team_df.AwayWinStreak
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin'])#,'DayofYear'])
df_noNa_div_date_lastMatch_streak = compute_winstreak(df_noNa_div_date_lastMatch)
df_noNa_div_date_lastMatch_streak

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,...,Year,Month,Sin_Month,Cos_Month,DayofYear,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak
0,Auxerre,Nice,1.0,2.0,A,1.727,3.100,4.500,-0.75,2.050,...,2003,2,0.866025,0.500000,33,0.538005,0.842942,3,0,0
1,Guingamp,Marseille,0.0,1.0,A,2.500,2.875,2.625,0.00,1.925,...,2003,2,0.866025,0.500000,33,0.538005,0.842942,3,0,0
2,Hamburg,Hannover,0.0,3.0,A,1.571,3.500,5.000,-0.75,1.800,...,2003,2,0.866025,0.500000,33,0.538005,0.842942,3,0,0
3,Hertha,Werder Bremen,0.0,3.0,A,1.833,3.200,3.750,-0.75,2.025,...,2003,2,0.866025,0.500000,33,0.538005,0.842942,3,0,0
4,Lens,Le Mans,0.0,0.0,D,1.571,3.250,5.500,-0.75,1.900,...,2003,2,0.866025,0.500000,33,0.538005,0.842942,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,Genoa,Milan,1.0,2.0,A,4.750,3.700,1.800,0.75,1.800,...,2024,11,-0.500000,0.866025,310,-0.811539,0.584298,1,0,2
38511,Girona,Mallorca,1.0,0.0,H,2.000,3.100,4.200,-0.50,2.050,...,2024,11,-0.500000,0.866025,310,-0.811539,0.584298,2,0,2
38512,Wolfsburg,Hoffenheim,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,2.080,...,2024,11,-0.500000,0.866025,314,-0.769415,0.638749,0,0,1
38513,Milan,Bologna,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,2.020,...,2024,11,-0.500000,0.866025,314,-0.769415,0.638749,0,0,0


In [356]:
def compute_winstodate(df):
    
    years = df.Year.unique()
    df_lst = []    
    for year in years:
        
        year_df = df[df.Year == year]
        year_df['HomeWin'] = year_df.Result.replace([0, 1, 2], [0, 0, 1])
        year_df['AwayWin'] = year_df.Result.replace([0, 1, 2], [1, 0, 0])
        year_df['HomeWinsToDate'] = None
        year_df['AwayWinsToDate'] = None
        
        hometeams = year_df.HomeTeam.unique()
        awayteams = year_df.AwayTeam.unique()
        # if year > 2024:
        #     # 将 AwayWin = 3 当作 0 来处理，fangzhi计算连胜记录
        #     year_df['HomeWin'] = year_df['HomeWin'].replace(3, 0)
        #     year_df['AwayWin'] = year_df['AwayWin'].replace(3, 0)
        for team in hometeams:
            team_df = year_df[(year_df.HomeTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))

             # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['HomeWinsToDate'] = team_df.HomeWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['HomeWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
            # team_df['HomeWinsToDate'] = team_df.HomeWin.cumsum()
            # year_df.loc[team_df.index, 'HomeWinsToDate'] = team_df.HomeWinsToDate
            
        for team in awayteams:
            # if team == 'Arsenal' and year == 2025:
            #     aa = 1
            team_df = year_df[(year_df.AwayTeam == team)]
            team_df = team_df.sort_values(['Year', 'DayofYear'], ascending = (True, True))
            
            # 计算截至当前场次之前的累计胜利次数（不包含当前场次）
            team_df['AwayWinsToDate'] = team_df.AwayWin.shift(1).cumsum()
            # 填充 NaN 值为 0，因为第一场比赛没有上一场比赛的数据
            team_df['AwayWinsToDate'].fillna(0, inplace=True)
            # 将更新后的数据回写到原 DataFrame
            year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            # team_df['AwayWinsToDate'] = team_df.AwayWin.cumsum()
            # year_df.loc[team_df.index, 'AwayWinsToDate'] = team_df.AwayWinsToDate
            
        df_lst.append(year_df)
        
    return pd.concat(df_lst, axis = 0).drop(columns = ['HomeWin', 'AwayWin','DayofYear'])
df_noNa_div_date_lastMatch_streak_windate = compute_winstodate(df_noNa_div_date_lastMatch_streak)
df_noNa_div_date_lastMatch_streak_windate.drop(columns = ['HomeTeam', 'AwayTeam'], inplace = True)
df_noNa_div_date_lastMatch_streak_windate

,FTHG,FTAG,FTR,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.0,2.0,A,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
1,0.0,1.0,A,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
2,0.0,3.0,A,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
3,0.0,3.0,A,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
4,0.0,0.0,D,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,1.0,2.0,A,4.750,3.700,1.800,0.75,1.800,1.980,1656.094849,...,11,-0.500000,0.866025,-0.811539,0.584298,1,0,2,5.0,4.0
38511,1.0,0.0,H,2.000,3.100,4.200,-0.50,2.050,1.850,1652.915283,...,11,-0.500000,0.866025,-0.811539,0.584298,2,0,2,3.0,9.0
38512,NaN,NaN,NaN,2.300,3.600,2.800,-0.25,2.080,1.820,1638.784424,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,1,3.0,8.0
38513,NaN,NaN,NaN,2.300,3.400,3.000,-0.25,2.020,1.880,1789.167236,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,0,4.0,5.0


In [357]:
asia_feats = ['AHh', 'B365AHH', 'B365AHA']
betting_feats = ['B365H', 'B365D', 'B365A']
betting_feats

['B365H', 'B365D', 'B365A']

In [358]:
def compute_meanodds(df, betting_feats):
    """
    """
    home_odds = []
    away_odds = []
    draw_odds = []
    for odd in betting_feats:
        odd_type = odd[-1]
        if odd_type == 'H':
            home_odds.append(odd)
        elif odd_type == 'A':
            away_odds.append(odd)
        else:
            draw_odds.append(odd)
    avg_home_odds = df[home_odds].mean(axis=1)
    avg_away_odds = df[away_odds].mean(axis=1)
    avg_draw_odds = df[draw_odds].mean(axis=1)
    
    ordered_cols = ['HomeOdds', 'AwayOdds', 'DrawOdds'] + df.columns.tolist()
    
    df['HomeOdds'] = avg_home_odds
    df['AwayOdds'] = avg_away_odds
    df['DrawOdds'] = avg_draw_odds
    
    return df[ordered_cols]
df_odd = compute_meanodds(df_noNa_div_date_lastMatch_streak_windate, betting_feats)
# df_odd.drop(columns = ['WHH', 'WHD', 'WHA', 'HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG', 'FTR'], inplace = True)
df_odd.drop(columns = ['HomeOdds', 'AwayOdds', 'DrawOdds', 'FTHG', 'FTAG', 'FTR'], inplace = True)
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,1.727,3.100,4.500,-0.75,2.050,1.850,1702.604858,1611.196045,-1,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
1,2.500,2.875,2.625,0.00,1.925,1.975,1685.016113,1665.625732,0,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
2,1.571,3.500,5.000,-0.75,1.800,2.100,1718.566284,1649.805298,-1,1,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
3,1.833,3.200,3.750,-0.75,2.025,1.875,1719.916748,1692.120972,-1,1,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
4,1.571,3.250,5.500,-0.75,1.900,2.000,1697.354004,1539.958130,-1,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,4.750,3.700,1.800,0.75,1.800,1.980,1656.094849,1783.127075,1,0,...,11,-0.500000,0.866025,-0.811539,0.584298,1,0,2,5.0,4.0
38511,2.000,3.100,4.200,-0.50,2.050,1.850,1652.915283,1660.488281,-1,0,...,11,-0.500000,0.866025,-0.811539,0.584298,2,0,2,3.0,9.0
38512,2.300,3.600,2.800,-0.25,2.080,1.820,1638.784424,1608.671387,-1,1,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,1,3.0,8.0
38513,2.300,3.400,3.000,-0.25,2.020,1.880,1789.167236,1791.398926,-1,0,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,0,4.0,5.0


In [359]:
from sklearn.preprocessing import MinMaxScaler
# 加载保存的 MinMaxScaler
# minmax_scaler = joblib.load('minmax_scaler_2025_01_30_22_23_33.pkl')
# minmax_scaler = joblib.load('minmax_scaler_2025_01_31_00_44_24.pkl')
# minmax_scaler = joblib.load('minmax_scaler_2025_02_04_23_35_59.pkl')
minmax_scaler = joblib.load('minmax_scaler_2025_02_16_10_03_11.pkl')
# minmax_scaler = MinMaxScaler()
df_odd.loc[:, ['Year']] = minmax_scaler.transform(df_odd.loc[:, ['Year']])

from sklearn.preprocessing import StandardScaler
# std_scaler = joblib.load('std_scaler2025_01_30_22_23_34.pkl')
# std_scaler = joblib.load('std_scaler_2025_01_31_00_44_24.pkl')
# std_scaler = joblib.load('std_scaler_2025_02_04_23_35_59.pkl')
std_scaler = joblib.load('std_scaler_2025_02_16_10_03_11.pkl')
# std_scaler = StandardScaler()
# to_scale = ['HomeWinStreak','AwayWinStreak','HomeWinsToDate', 'AwayWinsToDate', 'HomeTeamELO', 'AwayTeamELO', 'HomeOdds', 'AwayOdds', 'DrawOdds'] + betting_feats
to_scale = ['HomeTeamELO', 'AwayTeamELO'] + betting_feats + asia_feats
df_odd.loc[:, to_scale] = std_scaler.transform(df_odd.loc[:, to_scale])
df_odd

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
0,-0.580526,-0.797610,0.067571,-0.569303,0.985962,-1.104719,0.064357,-0.690270,-1,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
1,-0.150674,-0.985651,-0.503986,0.336567,-0.295153,0.242156,-0.081397,-0.239874,0,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
2,-0.667275,-0.463316,0.219986,-0.569303,-1.576268,1.589031,0.196625,-0.370785,-1,1,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
3,-0.521581,-0.714037,-0.161052,-0.569303,0.729739,-0.835344,0.207816,-0.020631,-1,1,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
4,-0.667275,-0.672250,0.372402,-0.569303,-0.551376,0.511531,0.020844,-1.279751,-1,0,...,2,0.866025,0.500000,0.538005,0.842942,3,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38510,1.100512,-0.296169,-0.755471,1.242436,-1.576268,0.296031,-0.321061,0.732429,1,0,...,11,-0.500000,0.866025,-0.811539,0.584298,1,0,2,5.0,4.0
38511,-0.428716,-0.797610,-0.023878,-0.267346,0.985962,-1.104719,-0.347409,-0.282386,-1,0,...,11,-0.500000,0.866025,-0.811539,0.584298,2,0,2,3.0,9.0
38512,-0.261891,-0.379743,-0.450641,0.034610,1.293430,-1.427969,-0.464509,-0.711161,-1,1,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,1,3.0,8.0
38513,-0.261891,-0.546890,-0.389675,0.034610,0.678494,-0.781469,0.781679,0.800877,-1,0,...,11,-0.500000,0.866025,-0.769415,0.638749,0,0,0,4.0,5.0


In [360]:
# 提取预测数据
testing_row_number = df_noNa_new.shape[0]
df_testing_data = df_odd.iloc[-testing_row_number:].loc[:, df_odd.columns != 'Result']
df_testing_data

,B365H,B365D,B365A,AHh,B365AHH,B365AHA,HomeTeamELO,AwayTeamELO,balance_val,Div 0,...,Month,Sin_Month,Cos_Month,Sin_Day,Cos_Day,Last Match Result,HomeWinStreak,AwayWinStreak,HomeWinsToDate,AwayWinsToDate
38512,-0.261891,-0.379743,-0.450641,0.03461,1.293430,-1.427969,-0.464509,-0.711161,-1,1,...,11,-0.5,0.866025,-0.769415,0.638749,0,0,1,3.0,8.0
38513,-0.261891,-0.546890,-0.389675,0.03461,0.678494,-0.781469,0.781679,0.800877,-1,0,...,11,-0.5,0.866025,-0.769415,0.638749,0,0,0,4.0,5.0
38514,-0.234087,-0.672250,-0.359192,0.03461,0.780984,-1.320219,-0.785259,-0.406187,-1,0,...,11,-0.5,0.866025,-0.769415,0.638749,0,0,0,4.0,10.0


In [361]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.init as init

# 定义 MLP 模型（与之前类似）
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_layers=[256, 128, 32], output_dim=3, dropout_prob=0.3):
        super(MLPModel, self).__init__()

        layers = []
        in_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(in_dim, hidden_dim))
            # self.initialize_weights(layers[-1], method='kaiming')  # 使用Kaiming初始化
            # layers.append(nn.BatchNorm1d(hidden_dim))  # 添加 BatchNorm 层
            layers.append(nn.ELU())
            layers.append(nn.Dropout(dropout_prob))  # 添加 Dropout 层
            in_dim = hidden_dim
            
        layers.append(nn.Linear(in_dim, output_dim))  # 输出层
        # self.initialize_weights(layers[-2], method='kaiming')  # 输出层初始化
        layers.append(nn.Softmax(dim=1))  # 归一化为概率分布
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)  # 将 softmax 归一化到 (0,2) 范围
    
    def initialize_weights(self, layer, method='kaiming'):
        """
        初始化网络的权重
        :param layer: 当前层
        :param method: 初始化方法 ('kaiming' 或 'xavier')
        """
        if isinstance(layer, nn.Linear):
            if method == 'kaiming':
                init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')  # for ELU/Relu
            elif method == 'xavier':
                init.xavier_normal_(layer.weight)  # for tanh or sigmoid activations
            if layer.bias is not None:
                init.zeros_(layer.bias)  # 初始化偏置为0
        elif isinstance(layer, nn.Conv1d):
            if method == 'kaiming':
                init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='relu')
            elif method == 'xavier':
                init.xavier_normal_(layer.weight)
            if layer.bias is not None:
                init.zeros_(layer.bias)  # Conv层的偏置初始化为0
                
# 计算 top-k 准确率
def compute_top_k_accuracy(model, x_test, y_test, k=2):
    model.eval()
    with torch.no_grad():
        outputs = model(x_test)
        _, top_k_indices = torch.topk(outputs, k, dim=1, largest=True, sorted=True)
        correct_top_1 = torch.sum(top_k_indices[:, 0] == y_test).item()
        correct_top_2 = torch.sum((top_k_indices[:, 0] == y_test) | (top_k_indices[:, 1] == y_test)).item()
        top_1_accuracy = correct_top_1 / len(y_test)
        top_2_accuracy = correct_top_2 / len(y_test)
    return top_1_accuracy, top_2_accuracy

In [362]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 设置浮点数打印精度，禁用科学计数法
torch.set_printoptions(precision=4, sci_mode=False)
# eval_model = torch.load("./MLPModel_2025_02_05_12_59_59.pt").to(device)
# eval_model = torch.load("./MLPModel_2025_02_07_12_11_26.pt").to(device)
# eval_model = torch.load("./MLPModel_2025_02_10_13_55_41.pt").to(device)
eval_model = torch.load("./MLPModel_2025_02_12_08_56_50.pt").to(device)
print(eval_model)
# 评估模型的 top-1 和 top-2 准确率
x_val = torch.tensor(df_testing_data.values.astype('float32'), dtype=torch.float32).to(device)
eval_model(x_val)

MLPModel(
  (network): Sequential(
    (0): Linear(in_features=439, out_features=128, bias=True)
    (1): ELU(alpha=1.0)
    (2): Dropout(p=0.6, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ELU(alpha=1.0)
    (5): Dropout(p=0.6, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ELU(alpha=1.0)
    (8): Dropout(p=0.6, inplace=False)
    (9): Linear(in_features=32, out_features=16, bias=True)
    (10): ELU(alpha=1.0)
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=16, out_features=3, bias=True)
    (13): Softmax(dim=1)
  )
)


tensor([[0.6265, 0.2252, 0.1484],
        [0.5330, 0.2035, 0.2635],
        [0.5443, 0.2670, 0.1887]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# 方法 1：直接创建
numbers = [0,2,0,2,0,0,1,2,1,0,1,2,2,0,1,2,1,1,1,0,1,]
y_val_fix = torch.tensor(numbers).to(device)
top_1_accuracy, top_2_accuracy = compute_top_k_accuracy(eval_model, x_val, y_val_fix, k=2)
print(f"Top-1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top-2 Accuracy: {top_2_accuracy:.4f}")

### 加载模型
# model_name = 'sklearn_mlp_' + local_time + '.joblib'
# model_name = './' + 'sklearn_svm_2025_01_27_16_48_56.joblib'
# model_name = './' + 'sklearn_svm_randsearch_2025_01_27_17_09_20.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_28_21_56_17.joblib'
# model_name = './' + 'sklearn_mlp_2025_01_29_15_12_48.joblib'
model_name = './' + 'sklearn_mlp_2025_01_31_10_13_42.joblib'
mlp = load(model_name)
print(mlp)
predict_val = mlp.predict(df_testing_data)
series_pre = pd.Series(predict_val, name='Predicted')
series_pre
# compare_result = pd.concat([series_pre, yTe.reset_index()], axis=1)
# compare_result